<a href="https://colab.research.google.com/github/anandraiyer/access_forums_eval/blob/main/testing_framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import math
from itertools import combinations
from itertools import chain
from itertools import product
from itertools import starmap
from functools import partial
import networkx as nx

In [2]:
#The ground truth is stored here
data = pd.read_csv('/content/drive/MyDrive/final.csv')

In [3]:
data.columns

Index(['Thread', 'DateTime', 'Author', 'Post', 'ParentPosts', 'PostID',
       'ThreadID', 'AuthorID', 'OriginID', 'DialogAct', 'ParentID_List'],
      dtype='object')

In [4]:
df = data[['Thread','ThreadID','PostID','DateTime','Author','Post','ParentID_List']]

In [5]:
df.head()

,Thread,ThreadID,PostID,DateTime,Author,Post,ParentID_List
0,Testing the new Site!,1,0,2015-11-30 11:59:19.544597,Tim Ford,Thank you very much for all those who worked o...,-1
1,Netflix not accessible to blind people using a...,2,1,2015-11-30 12:05:11.019288,Tim Ford,"Hi All, For those out there who want to use N...",-1
2,Testing the new Site!,1,2,2015-11-30 12:06:01.976409,"Walker, Michael E","Hi Tim, the group is working fine. I got your ...",[0]
3,Testing the new Site!,1,3,2015-11-30 12:12:07.800324,ken lawrence,Should the JDH mail be deleted?,[0]
4,Netflix not accessible to blind people using a...,2,4,2015-11-30 12:14:27.873186,Greg Nickel,Will do…,[1]


In [6]:
def get_conversation_dag(tid):
  #Returns the Directed Acyclic Graph representing the Thread. Here nodes are messages and edges capture the 'reply to' relationship.
  #Use this function to calculate metrics using networkx library on graphs downstream.
  temp = df[df['ThreadID']==tid]

  threads = list(temp['Thread'])
  postids = list(temp['PostID'])
  posts = list(temp['Post'])
  datetimes = list(temp['DateTime'])
  authors = list(temp['Author'])
  parentids = list(temp['ParentID_List'])

  G = nx.DiGraph()
  G.add_nodes_from(posts)

  edges = []
  for i in range(len(postids)):
    parent = parentids[i]
    child = postids[i]

    if parent == "-1":
      continue
    else:
      if parent == "[]":
        if i > 1:
          parent = "["+str(postids[i-1])+"]"
        else:
          parent = "["+str(postids[0])+"]"
      parent = int(parent[1:-1])
    edges.append((parent,child))
  G.add_edges_from(edges)
  return G

In [7]:
def get_subthreads(tid):
  #Get a list of lists. Given a thread id get all the subthreads.
  temp = df[df['ThreadID']==tid]

  threads = list(temp['Thread'])
  postids = list(temp['PostID'])
  if len(postids)<2:
    return (postids)
  posts = list(temp['Post'])
  datetimes = list(temp['DateTime'])
  authors = list(temp['Author'])
  parentids = list(temp['ParentID_List'])

  G = nx.DiGraph()
  G.add_nodes_from(posts)

  edges = []
  for i in range(len(postids)):
    parent = parentids[i]
    child = postids[i]

    if parent == "-1":
      continue  
    else:
      if parent == "[]":
        if i > 1:
          parent = "["+str(postids[i-1])+"]"
        else:
          parent = "["+str(postids[0])+"]"
      parent = int(parent[1:-1])
    edges.append((parent,child))
  G.add_edges_from(edges)
  chaini = chain.from_iterable
  roots = (n for n,d in G.in_degree() if d==0)
  leaves = (n for n,d in G.out_degree() if d==0)
  all_paths = partial(nx.all_simple_paths, G)
  ans = list(chaini(starmap(all_paths, product(roots, leaves))))
  return(ans)

In [8]:
def get_list_subthreads(cluster_name,threadID):
  conversations = []
  for i, val in enumerate(get_subthreads(threadID)):
    conversations.append(cluster_name+str(i)+":"+str(val)[1:-1].replace(',',''))
  return conversations

In [9]:
def create_contingency_table(gold, auto):
  #Refer https://en.wikipedia.org/wiki/Rand_index#The_contingency_table
  table = []
  names_gold = []
  names_auto = []
  for i, _ in gold.items():
    names_gold.append(i)
  for i, _ in auto.items():
    names_auto.append(i)
  for i, v1 in gold.items():
    table_row = []
    for j, v2 in auto.items():
      table_row.append(len(v1.intersection(v2)))
    table.append(table_row)
  table = np.array(table)
  sum_rows = np.sum(table, axis = 1)
  sum_cols = np.sum(table, axis = 0)
  return table, sum_rows, sum_cols, names_gold, names_auto

In [10]:
def get_n(s1):
  #size of dataset D. Total number of posts in a thread.
  u = set([])
  for _,v in s1.items():
    u = u.union(set(v))
  return(len(u))

In [11]:
def get_length_clustering(s):
  #Get length of each clustering within a cluster s = {s1,s2,s3}, here we return [len(s1),len(s2)...]
  list_len = []
  for _,v in s.items():
    list_len.append(len(v))
  return(list_len)

In [12]:
def get_points(s1):
  #Get all points in the dataset by union of u and v in cluster (set of sets) S1
  u = set([])
  for _,v in s1.items():
    u = u.union(set(v))
  return(u)

In [13]:
def get_vi(s1,s2):
  #Refer : https://sites.stat.washington.edu/mmp/Papers/compare-colt.pdf
  c,c_rows,c_cols,g_name, a_name = create_contingency_table(s1,s2)
  #print(c)
  N = get_n(s1)
  
  H_uv = 0.0
  I_uv = 0.0

  X = get_length_clustering(s1)
  Y = get_length_clustering(s2)
  total = N
  for i in range(len(c_rows)):
    for j in range(len(c_cols)):
      if c[i][j] != 0:
        num = c[i][j]
        A = c[i][j] / X[i]
        B = c[i][j] / Y[j]
        H_uv = H_uv - ((num / total) * math.log(num / total, 2.0))
        I_uv = I_uv + ((num / total) * math.log((num * total) / (X[i] * Y[j]), 2.0))
      else:
        continue
  max_score = math.log(total, 2.0)
  VI = H_uv - I_uv
    
  scaled_VI = VI / max_score
  #print(sum_vi,math.log(total,2),scaled_vi,1-scaled_vi)
  return round((1 - scaled_VI),3)

In [14]:
def get_one_to_one(s1,s2):
  #https://aclanthology.org/P08-1095.pdf 
  contingency,rows_sums,col_sums,gold_name,auto_name = create_contingency_table(s1,s2)
  X = get_length_clustering(s1)
  Y = get_length_clustering(s2)
  N = get_n(s1)
  B = nx.Graph()
  left_nodes = []
  for i in gold_name:
    left_nodes.append('Left_'+str(i))
  right_nodes = []
  for i in auto_name:
    right_nodes.append('Right_'+str(i))
  B.add_nodes_from(left_nodes, bipartite=0)
  B.add_nodes_from(right_nodes, bipartite=1)

  for i in range(len(X)):
    for j in range(len(Y)):
      B.add_edge(left_nodes[i], right_nodes[j], weight = contingency[i][j])
  
  matches = nx.algorithms.matching.max_weight_matching(B)
  one_one_ratio = 0.0
  for u,v in matches:
    one_one_ratio = one_one_ratio + (100.0 * B.get_edge_data(u,v)['weight']/N)
  return(round(one_one_ratio,3))

In [15]:
def get_omega_score(s1,s2):
  #Refer : https://iopscience.iop.org/article/10.1088/1742-5468/2011/02/P02017/pdf (pg 6)
  n = get_n(s1)
  N = n*(n-1.0)/2.0
  all_points = get_points(s1)
  all_pairs = combinations(all_points,2)
  s1_count = {}
  s2_count = {}
  for i,j in all_pairs:
    for m,n in s1.items():
      if i in n and j in n:
        if (i,j) in s1_count:
          s1_count[(i,j)] = s1_count.get((i,j)) + 1
        else:
          s1_count[(i,j)] = 1
      else:
        if (i,j) in s1_count:
          s1_count[(i,j)] = s1_count.get((i,j)) + 0
        else:
          s1_count[(i,j)] = 0
    for m,n in s2.items():
      if i in n and j in n:
        if (i,j) in s2_count:
          s2_count[(i,j)] = s2_count.get((i,j)) + 1
        else:
          s2_count[(i,j)] = 1
      else:
        if (i,j) in s2_count:
          s2_count[(i,j)] = s2_count.get((i,j)) + 0
        else:
          s2_count[(i,j)] = 0
  
  count_s1_rev = {}
  count_s2_rev = {}
  for (i,j),k in s1_count.items():
    if count_s1_rev.get((k),-1)!=-1:
      count_s1_rev[k] = count_s1_rev.get(k).union(set([(i,j)]))
    else:
      count_s1_rev[k] = set([(i,j)])
  for (i,j),k in s2_count.items():
    if count_s2_rev.get((k),-1)!=-1:
      count_s2_rev[k] = count_s2_rev.get(k).union(set([(i,j)]))
    else:
      count_s2_rev[k] = set([(i,j)])
  
  i_max = max(list(count_s1_rev.keys()))
  j_max = max(list(count_s2_rev.keys()))
  min_ij = min(i_max,j_max)

  count_s1_s2_rev = {}
  for i in range(0,min_ij+1):
    inter = count_s1_rev.get(i,set([])).intersection(count_s2_rev.get(i,set([])))
    count_s1_s2_rev[i] = inter

  U_sum = 0.0
  for i in range(0,min_ij+1):
    U_sum = U_sum + len(count_s1_s2_rev.get(i,set([])))
  U_sum = U_sum / N
  E_sum = 0.0
  for i in range(0,min_ij+1):
    E_sum = E_sum + (len(count_s1_rev.get(i,set([])))*len(count_s2_rev.get(i,set([]))))
  E_sum = E_sum / (N * N)
  
  omega = (U_sum - E_sum)/(1.0- E_sum)
  return(round(omega,3))

In [16]:
def create_shen_precision_table(s1, s2):
  #Definition : https://www.microsoft.com/en-us/research/wp-content/uploads/2006/01/p35-shen.pdf (pg 40)
  table = []

  X = get_length_clustering(s1)
  Y = get_length_clustering(s2)

  a = 0
  b = 0
  for i, v1 in s1.items():
    table_row = []
    n_i = X[a]
    b = 0
    for j, v2 in s2.items():
      n_j = Y[b]
      if(n_j == 0):
        table_row.append(0)
      else:
        table_row.append(len(v1.intersection(v2))/n_j)
      b = b+1
    table.append(table_row)
    a=a+1
  table = np.array(table)
  
  return table

In [17]:
def create_shen_recall_table(s1, s2):
  #Definition : https://www.microsoft.com/en-us/research/wp-content/uploads/2006/01/p35-shen.pdf (pg 40)
  table = []
  
  X = get_length_clustering(s1)
  Y = get_length_clustering(s2)

  a = 0
  b = 0
  for i, v1 in s1.items():
    table_row = []
    n_i = X[a]
    b = 0
    for j, v2 in s2.items():
      n_j = Y[b]
      if n_i == 0:
        table_row.append(0)
      else:
        table_row.append(len(v1.intersection(v2))/n_i)
      b = b+1
    table.append(table_row)
    a=a+1
  table = np.array(table)
  
  return table

In [18]:
def create_shen_F_table(s1, s2):
  #Definition : https://www.microsoft.com/en-us/research/wp-content/uploads/2006/01/p35-shen.pdf (pg 40)
  prec_table = create_shen_precision_table(s1,s2)
  recall_table = create_shen_recall_table(s1,s2)
 
  X = get_length_clustering(s1)
  Y = get_length_clustering(s2)

  a = 0
  b = 0
  max_f = []
  for i, v1 in s1.items():
    table_row = []
    n_i = X[a]
    b = 0
    for j, v2 in s2.items():
      n_j = Y[b]
      if(prec_table[a][b] + recall_table[a][b] > 0):
        f = (2 * prec_table[a][b] * recall_table[a][b])/(prec_table[a][b] + recall_table[a][b])
      else:
        f = 0
      table_row.append(f)
      b = b+1
    #Select Max_{j} F(i,j) for every row i
    max_f.append(max(table_row))
    a=a+1

  return max_f

In [19]:
def get_shen_f1(s1,s2):
  #Refer : https://github.com/jkkummerfeld/irc-disentanglement/blob/master/tools/evaluation/conversation-eval.py
  #Definition : https://www.microsoft.com/en-us/research/wp-content/uploads/2006/01/p35-shen.pdf (pg 40)
  max_f_table = create_shen_F_table(s1,s2)
  sum_f = 0.0
  n = get_n(s1)
  X = get_length_clustering(s1)

  for i in range(len(max_f_table)):
    sum_f = sum_f + ( (X[i]/n) * max_f_table[i])
  return(round(sum_f,3))

In [20]:
gold = {'C0':{1,2,3,4},'C1':{5,6}}
auto = {'C0':{1,2,3,4},'C1':{5,6}}
VI = get_vi(gold, auto)
one_one = get_one_to_one(gold,auto)
omega = get_omega_score(gold,auto)
shen_f1 = get_shen_f1(gold,auto)

print(VI, one_one, omega,shen_f1)
#Sanity check 

0.0 100.0 1.0 1.0


In [21]:
model1_test = pd.read_csv('/content/drive/MyDrive/inference.forum.test.out', header=None) #2019 ACL Model by Kummerfeld et al. trained on IRC
model2_test = pd.read_csv('/content/drive/MyDrive/inference.forum.test.1.out', header=None) #2019 ACL Model by Kummerfeld et al. trained on Forum
model3_test = pd.read_csv('/content/drive/MyDrive/inference-forum.ptr.out', header=None) #2020 EMNLP Model by Yu et al. trained on IRC

In [22]:
threadID_list = []
parent_list = []
child_list = []
for i in list(model1_test[0]):
  if i[0]=='#':
    continue
  else:
    path = i.split(':')
    thread = path[0].split('.')
    threadID = thread[3]
    anno = path[1].split(' ')
    parent = anno[1]
    child = anno[0]
    threadID_list.append(threadID)
    parent_list.append(parent)
    child_list.append(child)
preds = pd.DataFrame()
preds['ThreadID'] = threadID_list
preds['P'] = parent_list
preds['C'] = child_list
preds = preds[preds['P']!=preds['C']]
preds = preds.sort_values(['ThreadID','P','C'])
preds = preds.reset_index(drop=True)

In [23]:
preds #predictions by model1 (Kummerfeld trained on IRC, inference prediction on our Forum test dataset) -> post id is relative for the thread

,ThreadID,P,C
0,100,10,11
1,100,14,18
2,100,15,16
3,100,2,9
4,100,7,10
...,...,...,...
10679,988,2,6
10680,988,9,11
10681,992,0,2
10682,992,2,3


In [24]:
threadID2_list = []
parent2_list = []
child2_list = []
for i in list(model2_test[0]):
  if i[0]=='#':
    continue
  else:
    path = i.split(':')
    thread = path[0].split('.')
    threadID = thread[3]
    anno = path[1].split(' ')
    parent = anno[1]
    child = anno[0]
    threadID2_list.append(threadID)
    parent2_list.append(parent)
    child2_list.append(child)
preds2 = pd.DataFrame()
preds2['ThreadID'] = threadID2_list
preds2['P'] = parent2_list
preds2['C'] = child2_list
preds2 = preds2[preds2['P']!=preds2['C']]
preds2 = preds2.sort_values(['ThreadID','P','C'])
preds2 = preds2.reset_index(drop=True)

In [25]:
preds2 #predictions by model1 (Kummerfeld trained on Forum dataset, inference prediction on our Forum test dataset) -> post id is relative for the thread

,ThreadID,P,C
0,100,0,1
1,100,1,2
2,100,1,4
3,100,10,11
4,100,11,12
...,...,...,...
19061,992,5,10
19062,992,5,6
19063,992,5,7
19064,992,5,9


In [35]:
model3_test

,0
0,# Fri Sep 24 19:28:46 2021
1,# disentangle.py inference-forum --model ./dat...
2,#isolate_count:0
3,#start_conv_count:0
4,# 0 19020
...,...
19020,./data/forum/test/2019-06-18.22614.annotation....
19021,./data/forum/test/2019-06-18.22614.annotation....
19022,./data/forum/test/2019-06-18.22614.annotation....
19023,./data/forum/test/2019-06-18.22614.annotation....


In [36]:
threadID3_list = []
parent3_list = []
child3_list = []
for i in list(model3_test[0]):
  if i[0]=='#':
    continue
  else:
    path = i.split(':')
    thread = path[0].split('.')
    threadID = thread[2]
    anno = path[1].split(' ')
    parent = anno[1]
    child = anno[0]
    threadID3_list.append(threadID)
    parent3_list.append(parent)
    child3_list.append(child)
preds3 = pd.DataFrame()
preds3['ThreadID'] = threadID3_list
preds3['P'] = parent3_list
preds3['C'] = child3_list
preds3 = preds3[preds2['P']!=preds2['C']]
preds3 = preds3.sort_values(['ThreadID','P','C'])
preds3 = preds3.reset_index(drop=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [37]:
preds3 #predictions on Pointer Network model by Yu et al. 

,ThreadID,P,C
0,100,0,1
1,100,0,9
2,100,10,10
3,100,10,11
4,100,11,13
...,...,...,...
19015,992,4,5
19016,992,4,7
19017,992,6,6
19018,992,8,8


In [38]:
#1000 uniformly random chosen threads with atleast 10 messages / posts.
test_threads = set([66,69,73,79,83,84,93,100,109,113,115,129,134,135,140,147,148,150,154,158,164,166,173,179,180,198,206,223,224,226,228,255,259,260,274,279,284,301,304,306,311,328,336,348,366,367,371,383,384,405,406,407,410,417,420,427,429,436,442,453,459,460,466,476,500,506,507,512,515,516,521,531,532,542,554,556,559,564,569,578,586,589,594,596,600,603,611,621,623,625,629,632,642,644,650,651,652,654,661,680,687,688,699,701,713,715,717,723,726,742,743,750,769,772,785,788,791,794,809,816,818,821,828,829,833,842,845,846,854,856,865,873,881,885,891,896,901,903,910,912,913,915,919,925,935,939,953,960,973,983,985,988,992,1021,1025,1031,1037,1056,1073,1106,1107,1117,1119,1124,1192,1197,1216,1246,1249,1275,1287,1305,1327,1346,1349,1357,1406,1408,1410,1412,1415,1434,1452,1486,1508,1514,1518,1530,1566,1584,1595,1597,1598,1606,1618,1631,1632,1653,1670,1682,1683,1686,1700,1736,1759,1762,1776,1777,1778,1779,1795,1802,1803,1810,1815,1826,1829,1834,1839,1843,1846,1858,1863,1867,1876,1889,1893,1894,1895,1904,1908,1912,1913,1915,1923,1931,1938,1944,1948,1949,1950,1954,1955,1976,1984,1987,1993,1999,2001,2014,2017,2018,2023,2025,2036,2046,2047,2052,2063,2065,2070,2073,2076,2094,2099,2119,2140,2148,2180,2183,2184,2202,2203,2208,2209,2215,2224,2228,2229,2245,2274,2294,2307,2318,2319,2335,2344,2347,2354,2358,2359,2372,2378,2383,2396,2398,2400,2402,2409,2410,2415,2416,2422,2430,2436,2446,2447,2472,2494,2499,2500,2503,2517,2521,2535,2558,2561,2577,2578,2597,2604,2612,2614,2616,2617,2618,2625,2637,2653,2666,2689,2741,2761,2762,2767,2773,2775,2785,2789,2799,2800,2824,2836,2837,2840,2851,2852,2853,2854,2863,2864,2870,2875,2883,2885,2893,2894,2896,2902,2912,2925,2927,2928,2932,2935,2942,2945,2946,2954,2966,2979,2982,2992,2996,3003,3004,3007,3015,3016,3040,3044,3047,3048,3050,3057,3058,3059,3060,3072,3090,3095,3107,3118,3125,3127,3128,3133,3137,3151,3158,3160,3166,3172,3205,3207,3215,3220,3221,3230,3235,3237,3275,3282,3286,3287,3289,3294,3330,3332,3333,3336,3344,3347,3360,3361,3378,3402,3415,3418,3420,3451,3453,3475,3476,3479,3489,3491,3517,3522,3525,3530,3542,3543,3549,3565,3570,3576,3593,3594,3612,3641,3644,3646,3655,3659,3668,3669,3679,3721,3722,3745,3752,3757,3758,3769,3772,3775,3796,3797,3799,3802,3806,3812,3819,3834,3838,3843,3847,3859,3866,3885,3889,3906,3908,3912,3915,3918,3919,3921,3923,3926,3927,3943,3946,3953,3963,3967,3970,3971,3973,12104,12107,12111,12113,12115,21159,12119,12120,12122,12124,12125,12127,21164,21166,21167,12133,12135,21170,21172,12137,12139,21176,21179,12159,21186,21189,12166,21194,21195,21197,12171,12174,21204,21207,21209,12181,12182,21212,21214,12191,21216,21217,21220,12196,21225,21228,21233,12203,12204,12206,21237,21241,12212,12213,12214,12216,12223,21262,21266,21269,21270,21271,12235,21274,21278,21288,12249,21295,21297,21299,12252,12258,12261,21321,21323,21330,21331,21332,21333,21339,21347,21350,21358,21364,21373,12301,21381,21383,12307,12317,21398,12318,12319,12320,12322,21404,21405,12327,21412,21414,12332,12335,21421,21426,21428,21440,21442,12346,21450,21457,12353,21465,21468,21469,21470,21474,12359,21475,21476,21481,21486,21493,21494,21497,21500,21503,21506,21507,12366,21511,21515,21517,21521,21527,21528,21532,21533,21535,21538,21541,21544,12390,21549,21550,21552,21556,21558,21564,12416,21567,21568,21569,12423,21583,21584,21592,21599,21602,12432,12436,21604,21605,21620,21623,21628,21631,21634,21638,21640,12452,21641,21642,21645,21647,12458,12460,21649,21650,21651,21652,21659,21664,12468,21668,21669,21673,21678,12475,21685,21689,21699,21704,21707,21709,12489,21719,21720,12494,12498,21731,12507,21738,21741,12512,12514,21745,21750,21755,21756,21758,21762,21766,21768,12518,21775,21778,21779,21780,21781,12523,21787,21788,21792,12528,21793,21797,12530,21798,12537,21803,12540,12541,21805,21808,21811,21813,12548,12549,21818,21821,21824,21828,21829,21832,21833,12562,21838,21839,21841,21848,21849,21853,21854,21855,21866,21868,12570,12571,12575,21878,12583,21884,21886,21889,21890,21891,12589,21893,21897,21898,12595,21905,21906,21907,21912,21914,21915,21925,21928,21930,12597,21935,21941,21942,21945,21949,21950,21954,21955,21956,21958,21959,21960,12611,12614,12615,21972,21975,21978,12617,21981,21982,21985,21993,12627,12632,21996,22000,12641,12642,22004,22006,22008,22009,22018,22019,22022,12654,22025,22028,22032,12656,22033,22036,22041,22042,22043,12664,22046,22047,22048,22052,22056,12671,12676,22064,22067,22068,12680,12682,12684,12687,22070,22071,22078,22081,12693,12694,22086,12701,22095,22102,22106,12713,22108,22110,22111,22112,22114,22117,22120,22121,22124,22131,22134,12723,22135,12726,22146,22157,12728,22163,22169,22172,22179,22181,22183,22194,22200,22203,12749,22216,22218,22219,22223,22227,12757,22230,22236,22237,22238,22241,22242,22243,22246,22249,22250,22255,22260,12772,22264,22267,12775,22272,22276,22277,22282,22283,22284,22286,12780,12781,22293,22305,22310,12785,22325,12791,12792,12794,22342,12796,22358,22359,22361,22362,22363,22370,22373,12815,22380,22382,12817,22386,22387,22388,12823,22391,22392,22393,22395,22396,22397,12824,12828,22403,12839,22406,22412,22413,12843,12845,22421,22423,22428,22429,12854,22443,12858,22445,12868,12869,12870,22466,22472,22484,22485,12889,22497,22498,22499,22500,22522,22524,22526,12912,22535,22540,22550,22556,22558,12928,12931,12933,22567,22568,22569,12938,22580,12941,22583,12952,12958,12960,22603,12961,12965,12967,12968,22614])

In [39]:
print(len(test_threads)) #Check 1000 threads
thread_list = []
parent_list = []
child_list = []
post_sn = {}
for i in test_threads:
  if i == '':
    continue
  else:
    
    temp = df[df['ThreadID']==int(i)]
    postIDs = sorted(temp['PostID'])
    
    k = 0
    for j in postIDs:
      post_sn[(int(i),int(j))] = int(k) #Dictionary to get relative post ID given thread ID and post ID 
      k = k + 1 
    s = get_conversation_dag(int(i))
    
    for j,k in s.edges():
      thread_list.append(int(i))

      p = post_sn.get((i,j))
      c = post_sn.get((i,k))

      parent_list.append(p)
      child_list.append(c)

1000


In [40]:
gold = pd.DataFrame()
gold['ThreadID'] = thread_list
gold['P'] = parent_list
gold['C'] = child_list
gold = gold[gold['P']!=gold['C']]
gold = gold.sort_values(['ThreadID','P','C'])
gold = gold.reset_index(drop=True)

In [41]:
gold #ground truth of forum test dataset

,ThreadID,P,C
0,66,0,1
1,66,0,3
2,66,1,2
3,66,1,5
4,66,3,4
...,...,...,...
19061,22614,17,18
19062,22614,17,23
19063,22614,19,20
19064,22614,19,22


In [42]:
new_tid = []
for i in preds['ThreadID']:
  new_tid.append(int(i))
preds['ThreadID'] = new_tid

In [43]:
new_tid = []
for i in preds2['ThreadID']:
  new_tid.append(int(i))
preds2['ThreadID'] = new_tid

In [44]:
new_tid = []
for i in preds3['ThreadID']:
  new_tid.append(int(i))
preds3['ThreadID'] = new_tid

In [45]:
def get_set_of_sets(s1):
  #Given a prediction, get a set of subthreads for each thread
  set_set_list = []
  for i in test_threads:
    temp = s1[s1['ThreadID']==int(i)]
    p = list(temp['P'])
    c = list(temp['C'])
    edges = []
    for j in range(len(p)):
      edges.append((int(p[j]),int(c[j])))
    all_nodes = list(set(p).union(set(c)))
    G = nx.DiGraph()
    G.add_nodes_from(all_nodes)
    G.add_edges_from(edges)
    chaini = chain.from_iterable
    roots = (n for n,d in G.in_degree() if d==0)
    leaves = (n for n,d in G.out_degree() if d==0)
    all_paths = partial(nx.all_simple_paths, G)
    ans = chaini(starmap(all_paths, product(roots, leaves)))
    set_ans = {}
    k = 0
    for i in ans:
      set_ans['C'+str(k)] = set(i)
      k = k + 1
    set_set_list.append(set_ans)
  return set_set_list

In [46]:
gt = get_set_of_sets(gold) #Ground Truth
pr = get_set_of_sets(preds) #Model 1
pr2 = get_set_of_sets(preds2) #Model 2
pr3 = get_set_of_sets(preds3) #Model 3

In [47]:
temp = pd.DataFrame()
temp['TID'] = list(test_threads)
temp['GT'] = gt
temp['PR'] = pr
temp['PR2'] = pr2
temp['PR3'] = pr3

In [67]:
preds3

,ThreadID,P,C
0,100,0,1
1,100,0,9
2,100,10,10
3,100,10,11
4,100,11,13
...,...,...,...
19015,992,4,5
19016,992,4,7
19017,992,6,6
19018,992,8,8


In [48]:
temp

,TID,GT,PR,PR2,PR3
0,2052,"{'C0': {0, 1}, 'C1': {0, 2, 3, 4, 5, 6, 7, 8},...","{'C0': {0, 2}, 'C1': {0, 4, 6}, 'C2': {0, 8, 4...","{'C0': {0, 1, 2}, 'C1': {0, 11, 3}, 'C2': {0, ...","{'C0': {0, 1}, 'C1': {0, 2}, 'C2': {0, 4, 5, 6..."
1,22535,"{'C0': {0, 1, 2, 3, 4, 5, 6}, 'C1': {0, 1, 2, ...","{'C0': {0, 2, 4}, 'C1': {0, 2, 6, 7, 9}, 'C2':...","{'C0': {0, 7}, 'C1': {0, 1, 2, 3, 4, 5, 6}, 'C...","{'C0': {0, 1}, 'C1': {0, 2}}"
2,22540,"{'C0': {0, 1}, 'C1': {0, 2}, 'C2': {0, 6}, 'C3...","{'C0': {0, 3}, 'C1': {0, 10, 11}, 'C2': {0, 10...","{'C0': {0, 13}, 'C1': {0, 1, 2}, 'C2': {0, 10,...","{'C0': {0, 3, 13}}"
3,12301,"{'C0': {0, 2}, 'C1': {0, 3}, 'C2': {0, 1, 4}, ...","{'C0': {0, 4}, 'C1': {11, 3, 12}, 'C2': {5, 6}...","{'C0': {0, 10}, 'C1': {0, 3}, 'C2': {0, 5}, 'C...",{}
4,2063,"{'C0': {0, 1, 2, 3, 4}, 'C1': {0, 5}, 'C2': {0...","{'C0': {0, 8, 2}, 'C1': {1, 3}, 'C2': {10, 12}...","{'C0': {0, 1}, 'C1': {0, 15}, 'C2': {0, 5}, 'C...","{'C0': {0, 8}, 'C1': {0, 2, 4}}"
...,...,...,...,...,...
995,2036,"{'C0': {0, 1, 2}, 'C1': {0, 1, 3}, 'C2': {0, 1...","{'C0': {1, 14}, 'C1': {19, 1, 18, 3}, 'C2': {2...","{'C0': {0, 1, 4, 5, 6, 9, 10, 11}, 'C1': {0, 1...","{'C0': {0, 1}}"
996,22522,"{'C0': {0, 1}, 'C1': {0, 2, 3, 4, 5}, 'C2': {0...","{'C0': {3, 5}, 'C1': {9, 6, 7}}","{'C0': {0, 1}, 'C1': {0, 2, 3, 4, 5, 6, 7}, 'C...",{}
997,22524,"{'C0': {0, 1, 2, 3, 4}, 'C1': {0, 5, 6}, 'C2':...","{'C0': {0, 2}, 'C1': {0, 4}, 'C2': {0, 1, 6, 7...","{'C0': {0, 19}, 'C1': {0, 5}, 'C2': {0, 7, 8, ...","{'C0': {0, 2}, 'C1': {0, 4}, 'C2': {0, 1, 6, 7..."
998,2046,"{'C0': {0, 1}, 'C1': {0, 2}, 'C2': {0, 3}, 'C3...","{'C0': {0, 11}, 'C1': {8, 1, 10}, 'C2': {2, 3}...","{'C0': {0, 1}, 'C1': {0, 23}, 'C2': {0, 33}, '...",{}


In [56]:
def get_metrics(s1,s2):
  VI_list = []
  omega_list = []
  one_one_list = []
  shen_f1_list = []
  counter = 0
  for i in range(len(s1)):
    if (len(s1[i]) == 0) or (len(s2[i]) == 0):
      counter = counter + 1
      VI_list.append(0)
      omega_list.append(0)
      one_one_list.append(0)
      shen_f1_list.append(0)
    else:
      VI = get_vi(s1[i],s2[i])
      OM = get_omega_score(s1[i],s2[i])
      OVO = get_one_to_one(s1[i],s2[i])
      SF1 = get_shen_f1(s1[i],s2[i])
      VI_list.append(VI)
      omega_list.append(OM)
      one_one_list.append(OVO)
      shen_f1_list.append(SF1)
  print('No Predictions :'+str(counter))
  metrics = pd.DataFrame()
  metrics['TID'] = list(test_threads)
  metrics['VI'] = VI_list
  metrics['Omega'] = omega_list
  metrics['one_to_one'] = one_one_list
  metrics['shen_F1'] = shen_f1_list
  return metrics

In [61]:
metrics = get_metrics(gt, pr)

No Predictions :7


In [62]:
metrics2 = get_metrics(gt, pr2)

No Predictions :0


In [63]:
metrics3 = get_metrics(gt, pr3)

No Predictions :311


In [64]:
metrics3.describe() #model3

,TID,VI,Omega,one_to_one,shen_F1
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,10570.267000,2.896322,0.062138,22.277984,0.674092
std,9136.185749,4.578787,0.083403,25.052731,0.517188
min,66.000000,0.000000,-0.116000,0.000000,0.000000
25%,2016.250000,0.000000,0.000000,0.000000,0.000000
50%,8038.500000,1.563985,0.032000,16.000000,0.775000
75%,21602.500000,3.729649,0.093250,33.333000,1.040500
max,22614.000000,71.423754,0.577000,142.500000,2.525000


In [65]:
metrics2.describe() #model 2

,TID,VI,Omega,one_to_one,shen_F1
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,10570.267000,21.566533,0.316285,122.850094,1.424375
std,9136.185749,27.804545,0.190010,27.833744,0.329036
min,66.000000,2.213854,-0.333000,45.455000,0.537000
25%,2016.250000,8.993516,0.187750,106.667000,1.210750
50%,8038.500000,12.942674,0.299000,120.000000,1.390500
75%,21602.500000,22.864126,0.430500,135.073500,1.580000
max,22614.000000,371.993184,1.000000,334.783000,3.927000


In [66]:
metrics.describe() #model 1

,TID,VI,Omega,one_to_one,shen_F1
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,10570.267000,5.791321,0.040897,50.798530,0.966490
std,9136.185749,4.589073,0.094611,18.236499,0.306344
min,66.000000,0.000000,-0.305000,0.000000,0.000000
25%,2016.250000,3.013670,-0.016250,38.889000,0.782000
50%,8038.500000,4.523089,0.033500,50.000000,0.942000
75%,21602.500000,6.988888,0.091000,61.564500,1.130000
max,22614.000000,44.393783,0.564000,150.000000,2.298000


In [76]:
metrics[metrics['VI']>5]

,TID,VI,Omega,one_to_one,shen_F1
0,2052,5.526692,0.063,91.667,1.272
2,22540,7.281291,0.164,78.571,1.095
5,2065,11.663825,0.017,58.621,1.374
6,12307,17.371846,-0.037,52.273,0.996
7,2070,7.077270,0.034,63.158,1.135
...,...,...,...,...,...
993,2025,6.889261,0.029,47.368,0.920
994,22500,6.144821,0.009,50.000,0.970
995,2036,6.471145,0.029,55.556,0.953
997,22524,11.794742,0.139,71.429,1.095


In [74]:
len(preds[preds['ThreadID']==21905]['P'])

39

In [75]:
len(gold[gold['ThreadID']==21905]['P'])

46

In [73]:
math.log(46,2)

5.882643049361842